In [17]:
# Climate Indices Visualization Project - English Years
# Updated to use real-time data from NOAA sources:
# - PDO (Pacific Decadal Oscillation): NOAA ERSST v5
# - ONI (Oceanic Niño Index): NOAA Climate Prediction Center
# - DMI (Dipole Mode Index/IOD): NOAA ERSST v5
#
# Data sources:
# PDO: https://www.ncei.noaa.gov/pub/data/cmb/ersst/v5/index/ersst.v5.pdo.dat
# ONI: https://www.cpc.ncep.noaa.gov/data/indices/oni.ascii.txt  
# DMI: https://www.ncei.noaa.gov/pub/data/cmb/ersst/v5/index/ersst.v5.iod.dat
    
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [18]:
# Import climate indices data from NOAA online sources using manual parsing

import requests
from io import StringIO

try:
    # Load PDO data from NOAA ERSST v5
    print("Loading PDO data...")
    pdo_url = "https://www.ncei.noaa.gov/pub/data/cmb/ersst/v5/index/ersst.v5.pdo.dat"
    response = requests.get(pdo_url)
    lines = response.text.strip().split('\n')
    
    # Parse PDO data manually
    pdo_data = []
    for line in lines:
        if line.strip() and not line.startswith('#'):
            parts = line.split()
            if len(parts) >= 13:  # Year + 12 months
                try:
                    year = int(parts[0])
                    months = [float(val) if val not in ['99.99', '-99.99'] else np.nan for val in parts[1:13]]  # Handle missing/future data
                    for month_idx, value in enumerate(months):
                        # Include all values (including NaN) to maintain data structure
                            month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
                                         'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
                            pdo_data.append([year, month_names[month_idx], value])
                except (ValueError, IndexError):
                    continue
    
    # Create PDO DataFrame
    pdo = pd.DataFrame(pdo_data, columns=['Year', 'Month', 'Value'])
    print(f"✓ PDO data loaded successfully ({len(pdo)} records)")

    # Load ONI data from NOAA CPC
    print("Loading ONI data...")
    oni_url = "https://www.cpc.ncep.noaa.gov/data/indices/oni.ascii.txt"
    response = requests.get(oni_url)
    lines = response.text.strip().split('\n')
    
    # Parse ONI data manually
    oni_data = []
    for line in lines:
        if line.strip() and not line.startswith('#') and not line.startswith('SEAS'):
            parts = line.split()
            if len(parts) >= 4:  # SEAS YR TOTAL ANOM format
                try:
                    season = parts[0].strip()
                    year = int(parts[1])
                    total = float(parts[2])  # Not used but present in format
                    anom = float(parts[3])
                    # Only include valid seasons
                    valid_seasons = ['DJF', 'JFM', 'FMA', 'MAM', 'AMJ', 'MJJ', 'JJA', 'JAS', 'ASO', 'SON', 'OND', 'NDJ']
                    if season in valid_seasons:
                        oni_data.append([season, year, anom])
                except (ValueError, IndexError):
                    continue
    
    # Create ONI DataFrame
    oni = pd.DataFrame(oni_data, columns=['Season', 'Year', 'Value'])
    print(f"✓ ONI data loaded successfully ({len(oni)} records)")

    # Load DMI/IOD data from NOAA ERSST v5
    print("Loading DMI data...")
    dmi_url = "https://www.ncei.noaa.gov/pub/data/cmb/ersst/v5/index/ersst.v5.iod.dat"
    response = requests.get(dmi_url)
    lines = response.text.strip().split('\n')
    
    # Parse DMI data manually
    dmi_data = []
    for line in lines:
        if line.strip() and not line.startswith('#'):
            parts = line.split()
            if len(parts) >= 5:  # Year Month West East Diff
                try:
                    year = int(parts[0])
                    month = int(parts[1])
                    west = float(parts[2])
                    east = float(parts[3])
                    diff = float(parts[4])  # This is the DMI value
                    if diff != -99.99:  # Filter out missing values
                        dmi_data.append([year, month, west, east, diff])
                except (ValueError, IndexError):
                    continue
    
    # Create DMI DataFrame
    dmi = pd.DataFrame(dmi_data, columns=['Year', 'Month', 'West', 'East', 'Diff'])
    print(f"✓ DMI data loaded successfully ({len(dmi)} records)")
    
    print(f"\nData loading completed successfully!")
    print(f"PDO: {len(pdo)} records")
    print(f"ONI: {len(oni)} records") 
    print(f"DMI: {len(dmi)} records")
    
except Exception as e:
    print(f"Error loading data: {e}")
    print("Please check your internet connection and data source URLs")


Loading PDO data...
✓ PDO data loaded successfully (2064 records)
Loading ONI data...
✓ ONI data loaded successfully (904 records)
Loading DMI data...
✓ DMI data loaded successfully (2057 records)

Data loading completed successfully!
PDO: 2064 records
ONI: 904 records
DMI: 2057 records


In [19]:
# Clean ONI data
print("Cleaning ONI data...")

# ONI data format: Season Year Value
# Rename columns to match existing code
oni.columns = ['Seasonal', 'Year', 'Anom']
oni = oni[['Year', 'Seasonal', 'Anom']]

# Define the order of the categories
month_order_oni = ['DJF', 'JFM', 'FMA', 'MAM', 'AMJ', 'MJJ', 'JJA', 'JAS', 'ASO', 'SON', 'OND', 'NDJ']

# Convert the 'Seasonal' column to a categorical type with order
oni['Seasonal'] = pd.Categorical(oni['Seasonal'], categories=month_order_oni, ordered=True)

# Sorting DataFrame into a correct Year and Month order
oni.sort_values(by=['Year', 'Seasonal'], inplace=True)

# Keep English years (no conversion)
# Reset the index
oni = oni.reset_index(drop=True)

print("ONI data after cleaning:")
print(oni.head(10))


Cleaning ONI data...
ONI data after cleaning:
   Year Seasonal  Anom
0  1950      DJF -1.53
1  1950      JFM -1.34
2  1950      FMA -1.16
3  1950      MAM -1.18
4  1950      AMJ -1.07
5  1950      MJJ -0.85
6  1950      JJA -0.54
7  1950      JAS -0.42
8  1950      ASO -0.39
9  1950      SON -0.44


In [20]:
# PDO data cleaning
print("Cleaning PDO data...")

# Data is already in the correct format from previous step: Year, Month, Value
print("PDO data structure:")
print(pdo.head())
print("\nPDO columns:", pdo.columns.tolist())

# Define the order of the categories
month_order_pdo = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

# Convert the 'Month' column to a categorical type with order
pdo['Month'] = pd.Categorical(pdo['Month'], categories=month_order_pdo, ordered=True)

# Sorting DataFrame into a correct Year and Month order
pdo.sort_values(by=['Year', 'Month'], inplace=True)

# Reset the index
pdo = pdo.reset_index(drop=True)

month_map = {
    'Jan': '1', 'Feb': '2', 'Mar': '3', 'Apr': '4',
    'May': '5', 'Jun': '6', 'Jul': '7', 'Aug': '8',
    'Sep': '9', 'Oct': '10', 'Nov': '11', 'Dec': '12'
}

pdo['Month'] = pdo['Month'].map(month_map)

# Keep English years (no conversion)

print("PDO data after processing:")
print(pdo.head(12))


Cleaning PDO data...
PDO data structure:
   Year Month  Value
0  1854   Jan   0.11
1  1854   Feb  -0.24
2  1854   Mar  -0.40
3  1854   Apr  -0.44
4  1854   May  -0.54

PDO columns: ['Year', 'Month', 'Value']
PDO data after processing:
    Year Month  Value
0   1854     1   0.11
1   1854     2  -0.24
2   1854     3  -0.40
3   1854     4  -0.44
4   1854     5  -0.54
5   1854     6  -0.30
6   1854     7  -0.10
7   1854     8  -1.24
8   1854     9  -1.00
9   1854    10  -2.23
10  1854    11  -1.68
11  1854    12  -1.76


In [21]:
# Process DMI data (use Diff column which is West - East = DMI)
print("Processing DMI data...")

# Use only Year, Month, and Diff columns (Diff = DMI) - proper copy to avoid warnings
dmi_processed = dmi[['Year', 'Month', 'Diff']].copy()

# Convert data types
dmi_processed['Month'] = dmi_processed['Month'].astype(str)

# Define the order of the categories
month_order_dmi = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12']

# Convert the 'Month' column to a categorical type with order
dmi_processed['Month'] = pd.Categorical(dmi_processed['Month'], categories=month_order_dmi, ordered=True)

# Keep English years (no conversion)

# Sorting DataFrame into a correct Year and Month order
dmi_processed.sort_values(by=['Year', 'Month'], inplace=True)

# Reset the index
dmi_processed = dmi_processed.reset_index(drop=True)

# Replace the original dataframe
dmi = dmi_processed

print("DMI data cleaned successfully")
print(dmi.head(12))


Processing DMI data...
DMI data cleaned successfully
    Year Month  Diff
0   1854     1  0.55
1   1854     2  0.29
2   1854     3  0.87
3   1854     4  0.85
4   1854     5  0.93
5   1854     6 -0.06
6   1854     7 -0.48
7   1854     8 -0.56
8   1854     9 -0.48
9   1854    10 -0.70
10  1854    11 -0.90
11  1854    12 -0.12


In [22]:
# Find all years with complete data for all 3 indices (current + previous year)
print("Finding all years with complete data...")

# Get available years for each dataset
oni_years = set(oni['Year'].unique())
pdo_years = set(pdo['Year'].unique())
dmi_years = set(dmi['Year'].unique())

print(f"Available years:")
print(f"ONI: {min(oni_years)} to {max(oni_years)} ({len(oni_years)} years)")
print(f"PDO: {min(pdo_years)} to {max(pdo_years)} ({len(pdo_years)} years)")
print(f"DMI: {min(dmi_years)} to {max(dmi_years)} ({len(dmi_years)} years)")

# Find overlapping years where all 3 indices have data
common_years = oni_years.intersection(pdo_years).intersection(dmi_years)
print(f"\nYears with all 3 indices: {min(common_years)} to {max(common_years)} ({len(common_years)} years)")

# Find years where both current year AND previous year have data for all indices
valid_years = []
for year in sorted(common_years):
    prev_year = year - 1
    if prev_year in common_years:  # Previous year also has all 3 indices
        valid_years.append(year)

print(f"\nValid years for visualization (with previous year data): {len(valid_years)} years")
print(f"Range: {min(valid_years)} to {max(valid_years)}")
print(f"Years: {valid_years[:5]}...{valid_years[-5:] if len(valid_years) > 10 else valid_years[5:]}")

# Create output directory if it doesn't exist
import os
output_dir = r"D:\HII\climate_indicies_visualization\output"
os.makedirs(output_dir, exist_ok=True)
print(f"\nOutput directory: {output_dir}")
print(f"Will generate {len(valid_years)} visualization images")


Finding all years with complete data...
Available years:
ONI: 1950 to 2025 (76 years)
PDO: 1854 to 2025 (172 years)
DMI: 1854 to 2025 (172 years)

Years with all 3 indices: 1950 to 2025 (76 years)

Valid years for visualization (with previous year data): 75 years
Range: 1951 to 2025
Years: [1951, 1952, 1953, 1954, 1955]...[2021, 2022, 2023, 2024, 2025]

Output directory: D:\HII\climate_indicies_visualization\output
Will generate 75 visualization images


In [23]:
# Calculate dynamic y-axis ranges to accommodate all data points
print("Calculating dynamic y-axis ranges...")

# Get min/max values for each dataset (excluding NaN values)
oni_min = oni['Anom'].min()
oni_max = oni['Anom'].max()

pdo_min = pdo['Value'].dropna().min()  # Drop NaN values from 99.99 entries
pdo_max = pdo['Value'].dropna().max()

dmi_min = dmi['Diff'].min()
dmi_max = dmi['Diff'].max()

print(f"Data ranges:")
print(f"ONI: {oni_min:.2f} to {oni_max:.2f}")
print(f"PDO: {pdo_min:.2f} to {pdo_max:.2f}")
print(f"DMI: {dmi_min:.2f} to {dmi_max:.2f}")

# Find overall min/max across all datasets
overall_min = min(oni_min, pdo_min, dmi_min)
overall_max = max(oni_max, pdo_max, dmi_max)

print(f"Overall range: {overall_min:.2f} to {overall_max:.2f}")

# Add padding (10% of the range) to ensure no points touch the edges
data_range = overall_max - overall_min
padding = data_range * 0.1

y_min = overall_min - padding
y_max = overall_max + padding

# Ensure symmetric range around zero for better visual balance
max_abs = max(abs(y_min), abs(y_max))
y_min = -max_abs
y_max = max_abs

print(f"Adjusted y-axis range with padding: {y_min:.2f} to {y_max:.2f}")

# Store for use in plotting
y_limits = (y_min, y_max)


Calculating dynamic y-axis ranges...
Data ranges:
ONI: -2.03 to 2.64
PDO: -3.80 to 3.84
DMI: -2.04 to 2.35
Overall range: -3.80 to 3.84
Adjusted y-axis range with padding: -4.60 to 4.60


In [24]:
# **Generate Combined Climate Indices Visualizations for All Valid Years**

print("Starting batch visualization generation...")
print(f"Processing {len(valid_years)} years...")

successful_generations = 0
failed_generations = []

for idx, year in enumerate(valid_years, 1):
    try:
        print(f"\nProcessing {idx}/{len(valid_years)}: Year {year}")
        
        # Create a figure with 3 subplots stacked vertically
        fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(12, 18))
        plt.subplots_adjust(hspace=0.4)  # Add space between subplots

        # =========================
        # ONI PLOT (Top subplot)
        # =========================

        # Get all available data for both years and create full time range structure
        oni_prev_year = oni[oni['Year'] == year - 1].copy()
        oni_curr_year = oni[oni['Year'] == year].copy()

        # Create complete seasonal structure
        full_seasons = ['DJF', 'JFM', 'FMA', 'MAM', 'AMJ', 'MJJ', 'JJA', 'JAS', 'ASO', 'SON', 'OND', 'NDJ']

        # Build complete dataset with proper structure
        oni_complete = []

        # Add previous year data (12 seasons)
        for season in full_seasons:
            prev_data = oni_prev_year[oni_prev_year['Seasonal'] == season]
            if len(prev_data) > 0:
                oni_complete.append([year - 1, season, prev_data.iloc[0]['Anom']])
            else:
                oni_complete.append([year - 1, season, np.nan])

        # Add current year data (12 seasons)  
        for season in full_seasons:
            curr_data = oni_curr_year[oni_curr_year['Seasonal'] == season]
            if len(curr_data) > 0:
                oni_complete.append([year, season, curr_data.iloc[0]['Anom']])
            else:
                oni_complete.append([year, season, np.nan])

        oni_plot_data = pd.DataFrame(oni_complete, columns=['Year', 'Seasonal', 'Anom'])

        # Create x-axis positions and labels
        x_positions = list(range(24))  # 0-23 for 24 periods
        x_labels = full_seasons + full_seasons  # Previous year + current year seasons

        # Plot only non-NaN values but maintain x-axis structure
        valid_x = []
        valid_y = []

        for i, row in oni_plot_data.iterrows():
            if not pd.isna(row['Anom']):
                valid_x.append(i)
                valid_y.append(row['Anom'])

        ax1.plot(valid_x, valid_y, marker='o', linewidth=2)

        # Add horizontal lines
        ax1.axhline(y=0, color='black', linestyle='--')
        ax1.axhline(y=0.5, color='red', linestyle='dotted', linewidth=1.2)
        ax1.axhline(y=-0.5, color='red', linestyle='dotted', linewidth=1.2)

        # Set consistent axis limits and styling using dynamic range
        ax1.set_ylim(y_limits)  # Use calculated dynamic range
        ax1.set_xlim(-0.5, 23.5)  # Full range for 24 periods
        ax1.set_xticks(x_positions)
        ax1.set_xticklabels(x_labels, fontsize=11)
        ax1.tick_params(axis='y', labelsize=12)

        # Add visual separator at year change (between position 11 and 12)
        ax1.axvline(x=11.5, color='black', linestyle='dashed', linewidth=1)

        # Add shaded backgrounds
        ax1.axvspan(-0.5, 11.5, color='lightgray', alpha=0.3)
        ax1.axvspan(11.5, 23.5, color='lightblue', alpha=0.3)

        # Add year annotations (position at 90% of max y-value)
        text_y_pos = y_limits[1] * 0.9
        ax1.text(5.5, text_y_pos, f'{year - 1}', fontsize=15, color='black', fontweight='bold')
        ax1.text(17.5, text_y_pos, f'{year}', fontsize=15, color='black', fontweight='bold')

        # Add data labels for extreme values
        for i, value in zip(valid_x, valid_y):
            if value > 0.5:
                ax1.annotate(f'{value:.1f}', (i, value), textcoords="offset points", xytext=(0, 12),
                            ha='center', fontsize=12, color='red')
            elif value < -0.5:
                ax1.annotate(f'{value:.1f}', (i, value), textcoords="offset points", xytext=(0, -18),
                            ha='center', fontsize=12, color='blue')

        # Customize ONI subplot
        ax1.set_title('ONI', fontsize=18)
        ax1.set_xlabel('Season', fontsize=12)

        # =========================
        # PDO PLOT (Middle subplot)
        # =========================

        # Get all available data for both years, filtering out NaN values (from 99.99)
        pdo_prev_year = pdo[pdo['Year'] == year - 1].copy()
        pdo_curr_year = pdo[pdo['Year'] == year].copy()

        # Create complete monthly structure
        full_months = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12']

        # Build complete dataset with proper structure
        pdo_complete = []

        # Add previous year data (12 months)
        for month in full_months:
            prev_data = pdo_prev_year[pdo_prev_year['Month'] == month]
            if len(prev_data) > 0 and not pd.isna(prev_data.iloc[0]['Value']):
                pdo_complete.append([year - 1, month, prev_data.iloc[0]['Value']])
            else:
                pdo_complete.append([year - 1, month, np.nan])

        # Add current year data (12 months)
        for month in full_months:
            curr_data = pdo_curr_year[pdo_curr_year['Month'] == month]
            if len(curr_data) > 0 and not pd.isna(curr_data.iloc[0]['Value']):
                pdo_complete.append([year, month, curr_data.iloc[0]['Value']])
            else:
                pdo_complete.append([year, month, np.nan])

        pdo_plot_data = pd.DataFrame(pdo_complete, columns=['Year', 'Month', 'Value'])

        # Create x-axis positions and labels
        x_positions = list(range(24))  # 0-23 for 24 periods
        x_labels = full_months + full_months  # Previous year + current year months

        # Plot only non-NaN values but maintain x-axis structure
        valid_x = []
        valid_y = []

        for i, row in pdo_plot_data.iterrows():
            if not pd.isna(row['Value']):
                valid_x.append(i)
                valid_y.append(row['Value'])

        ax2.plot(valid_x, valid_y, marker='o', linewidth=2)

        # Add horizontal lines
        ax2.axhline(y=0, color='black', linestyle='--')
        ax2.axhline(y=0.5, color='red', linestyle='dotted', linewidth=1.2)
        ax2.axhline(y=-0.5, color='red', linestyle='dotted', linewidth=1.2)

        # Set consistent axis limits and styling using dynamic range
        ax2.set_ylim(y_limits)  # Use calculated dynamic range
        ax2.set_xlim(-0.5, 23.5)  # Full range for 24 periods
        ax2.set_xticks(x_positions)
        ax2.set_xticklabels(x_labels, fontsize=15)
        ax2.tick_params(axis='y', labelsize=12)

        # Add visual separator at year change (between position 11 and 12)
        ax2.axvline(x=11.5, color='black', linestyle='dashed', linewidth=1)

        # Add shaded backgrounds
        ax2.axvspan(-0.5, 11.5, color='lightgray', alpha=0.3)
        ax2.axvspan(11.5, 23.5, color='lightblue', alpha=0.3)

        # Add year annotations (position at 90% of max y-value)
        ax2.text(5.5, text_y_pos, f'{year - 1}', fontsize=15, color='black', fontweight='bold')
        ax2.text(17.5, text_y_pos, f'{year}', fontsize=15, color='black', fontweight='bold')

        # Add data labels for extreme values
        for i, value in zip(valid_x, valid_y):
            if value > 0.5:
                ax2.annotate(f'{value:.1f}', (i, value), textcoords="offset points", xytext=(0, 12),
                            ha='center', fontsize=12, color='red')
            elif value < -0.5:
                ax2.annotate(f'{value:.1f}', (i, value), textcoords="offset points", xytext=(0, -18),
                            ha='center', fontsize=12, color='blue')

        # Customize PDO subplot
        ax2.set_title('PDO', fontsize=18)
        ax2.set_xlabel('Month', fontsize=12)

        # =========================
        # DMI PLOT (Bottom subplot)
        # =========================

        # Get all available data for both years
        dmi_prev_year = dmi[dmi['Year'] == year - 1].copy()
        dmi_curr_year = dmi[dmi['Year'] == year].copy()

        # Create complete monthly structure
        full_months = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12']

        # Build complete dataset with proper structure
        dmi_complete = []

        # Add previous year data (12 months)
        for month in full_months:
            prev_data = dmi_prev_year[dmi_prev_year['Month'] == month]
            if len(prev_data) > 0:
                dmi_complete.append([year - 1, month, prev_data.iloc[0]['Diff']])
            else:
                dmi_complete.append([year - 1, month, np.nan])

        # Add current year data (12 months)
        for month in full_months:
            curr_data = dmi_curr_year[dmi_curr_year['Month'] == month]
            if len(curr_data) > 0:
                dmi_complete.append([year, month, curr_data.iloc[0]['Diff']])
            else:
                dmi_complete.append([year, month, np.nan])

        dmi_plot_data = pd.DataFrame(dmi_complete, columns=['Year', 'Month', 'Diff'])

        # Create x-axis positions and labels
        x_positions = list(range(24))  # 0-23 for 24 periods
        x_labels = full_months + full_months  # Previous year + current year months

        # Plot only non-NaN values but maintain x-axis structure
        valid_x = []
        valid_y = []

        for i, row in dmi_plot_data.iterrows():
            if not pd.isna(row['Diff']):
                valid_x.append(i)
                valid_y.append(row['Diff'])

        ax3.plot(valid_x, valid_y, marker='o', linewidth=2)

        # Add horizontal lines
        ax3.axhline(y=0, color='black', linestyle='--')
        ax3.axhline(y=0.5, color='red', linestyle='dotted', linewidth=1.2)
        ax3.axhline(y=-0.5, color='red', linestyle='dotted', linewidth=1.2)

        # Set consistent axis limits and styling using dynamic range
        ax3.set_ylim(y_limits)  # Use calculated dynamic range
        ax3.set_xlim(-0.5, 23.5)  # Full range for 24 periods
        ax3.set_xticks(x_positions)
        ax3.set_xticklabels(x_labels, fontsize=15)
        ax3.tick_params(axis='y', labelsize=12)

        # Add visual separator at year change (between position 11 and 12)
        ax3.axvline(x=11.5, color='black', linestyle='dashed', linewidth=1)

        # Add shaded backgrounds
        ax3.axvspan(-0.5, 11.5, color='lightgray', alpha=0.3)
        ax3.axvspan(11.5, 23.5, color='lightblue', alpha=0.3)

        # Add year annotations (position at 90% of max y-value)
        ax3.text(5.5, text_y_pos, f'{year - 1}', fontsize=15, color='black', fontweight='bold')
        ax3.text(17.5, text_y_pos, f'{year}', fontsize=15, color='black', fontweight='bold')

        # Add data labels for extreme values
        for i, value in zip(valid_x, valid_y):
            if value > 0.5:
                ax3.annotate(f'{value:.1f}', (i, value), textcoords="offset points", xytext=(0, 12),
                            ha='center', fontsize=12, color='red')
            elif value < -0.5:
                ax3.annotate(f'{value:.1f}', (i, value), textcoords="offset points", xytext=(0, -18),
                            ha='center', fontsize=12, color='blue')

        # Customize DMI subplot
        ax3.set_title('DMI', fontsize=18)
        ax3.set_xlabel('Month', fontsize=12)

        # =========================
        # FINAL ADJUSTMENTS
        # =========================

        # Apply tight layout
        plt.tight_layout()

        # Save the combined figure to output directory
        output_filename = os.path.join(output_dir, f'{year}.png')
        plt.savefig(output_filename, dpi=300, bbox_inches='tight')
        plt.close()  # Close the figure to free memory
        
        successful_generations += 1
        if idx % 10 == 0:  # Progress update every 10 images
            print(f"  ✓ Completed {idx}/{len(valid_years)} images")
        
    except Exception as e:
        print(f"  ✗ Error generating visualization for year {year}: {str(e)}")
        failed_generations.append((year, str(e)))
        plt.close()  # Ensure figure is closed even on error

print(f"\n" + "="*60)
print("BATCH GENERATION COMPLETED")
print(f"="*60)
print(f"✓ Successfully generated: {successful_generations} images")
print(f"✗ Failed generations: {len(failed_generations)}")
print(f"📁 Output directory: {output_dir}")

if failed_generations:
    print(f"\nFailed years:")
    for year, error in failed_generations[:5]:  # Show first 5 errors
        print(f"  - Year {year}: {error}")
    if len(failed_generations) > 5:
        print(f"  ... and {len(failed_generations) - 5} more")

print(f"\nAll visualization images have been saved to the output directory with English year naming.")


Starting batch visualization generation...
Processing 75 years...

Processing 1/75: Year 1951

Processing 2/75: Year 1952

Processing 3/75: Year 1953

Processing 4/75: Year 1954

Processing 5/75: Year 1955

Processing 6/75: Year 1956

Processing 7/75: Year 1957

Processing 8/75: Year 1958

Processing 9/75: Year 1959

Processing 10/75: Year 1960
  ✓ Completed 10/75 images

Processing 11/75: Year 1961

Processing 12/75: Year 1962

Processing 13/75: Year 1963

Processing 14/75: Year 1964

Processing 15/75: Year 1965

Processing 16/75: Year 1966

Processing 17/75: Year 1967

Processing 18/75: Year 1968

Processing 19/75: Year 1969

Processing 20/75: Year 1970
  ✓ Completed 20/75 images

Processing 21/75: Year 1971

Processing 22/75: Year 1972

Processing 23/75: Year 1973

Processing 24/75: Year 1974

Processing 25/75: Year 1975

Processing 26/75: Year 1976

Processing 27/75: Year 1977

Processing 28/75: Year 1978

Processing 29/75: Year 1979

Processing 30/75: Year 1980
  ✓ Completed 30/75